### **ML Bootcamp Lab**

#### Rachel Seo ydp7xv

Goal: Build two data prep pipelines using different datasets to get practice with data preparation and question building.

Step one: Review these two datasets and brainstorm problems that could be addressed with the dataset. Identify a question for each dataset. 

1. College Completion Data
   
    a. Can we predict high college completion rates? (classification)


2. Job Placement Data: 

    a. Can we predict if the candidate is a male? (classification)

[College Completion Data Dictionary + Data](https://www.kaggle.com/datasets/thedevastator/boost-student-success-with-college-completion-da/data)

  - Dataset is in the data file in the DS 3021 repo, but was added after class started so you may need to "git pull" to get it.

[Job Placement](https://raw.githubusercontent.com/DG1606/CMS-R-2020/master/Placement_Data_Full_Class.csv) 

 - [Data Dictionary (kinda) for Job Placement](https://www.kaggle.com/benroshan/factors-affecting-campus-placement/discussion/280612) - You'll need to infer from the column names but also the comments on the site.

Step two: Work through the steps outlined in the examples to include the following elements: 

  * What is a independent Business Metric for your problem? 

    a. College Completion: Assuming that higher ratings results in an **increase in university profit**, can we predict the likelihood of a student completing all 4 years at an institution at a certain threshold?
    
    b. Job Placement: Assuming that men have a **higher chance for job placement**, can we predict the likelihood of a candidate being male?
 
  * Data preparation:  
    * correct variable type/class as needed
    * collapse factor levels as needed
    * one-hot encoding factor variables 
    * normalize the continuous variables
    * drop unneeded variables
    * create target variable if needed
  * Calculate the prevalence of the target variable 
  * Create the necessary data partitions (Train,Tune,Test)

Step three: What do your instincts tell you about the data. Can it address your problem, what areas/items are you worried about? 

1. The college completion data tells me that my problem can be addressed because there is a column called 'retain_value'. I am worried about the sheer volume of colleges within the dataset and iterating through all of it.

2. The job placement data tells me that my problem can be addressed because the dataset shows education level, employability test percentages, sex categories. 

In [ ]:
# import
import pandas as pd
import numpy as np 

# install matplotlib
# %pip install matplotlib
import matplotlib as plt

#make sure to install sklearn in your terminal first!
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### College Completion Data 

In [1]:
# cc_institution data 

cc_institution = pd.read_csv("../data/cc_institution_details.csv")
cc_institution.info()

NameError: name 'pd' is not defined

**Correcting Variable Types**

In [ ]:
# categorical variables to 'category' dtype
categorical_cols = ["city", "state", "level", "control", "basic"]
cc_institution[categorical_cols] = cc_institution[categorical_cols].astype("category")

# binary variables to boolean (1/0)
binary_cols = ["hbcu", "flagship"]
cc_institution[binary_cols] = cc_institution[binary_cols].fillna("No") 
cc_institution[binary_cols] = cc_institution[binary_cols].applymap(lambda x: 1 if x == "Yes" else 0)

# split column 'counted_pct'
cc_institution[['counted_percent', 'counted_pct_category']] = cc_institution['counted_pct'].str.split('|', expand=True) 

# object to float
cc_institution['counted_percent'] = pd.to_numeric(cc_institution['counted_percent'], errors='coerce')

# 'vsa_year' to integer 
if 'vsa_year' in cc_institution.columns:
    cc_institution['vsa_year'] = cc_institution['vsa_year'].fillna(0).astype(int)  # fill missing with 0 before converting

# dropping unnecessary columns
cc_institution.drop(columns=["index"], inplace=True, errors='ignore')
cc_institution.drop(columns=["similar"], inplace=True, errors='ignore')
cc_institution.drop(columns=["counted_pct"], inplace=True, errors='ignore')

# updated
cc_institution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 62 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   unitid                                3798 non-null   int64   
 1   chronname                             3798 non-null   object  
 2   city                                  3798 non-null   category
 3   state                                 3798 non-null   category
 4   level                                 3798 non-null   category
 5   control                               3798 non-null   category
 6   basic                                 3798 non-null   category
 7   hbcu                                  3798 non-null   int64   
 8   flagship                              3798 non-null   int64   
 9   long_x                                3798 non-null   float64 
 10  lat_y                                 3798 non-null   float64 
 11  site

/tmp/ipykernel_66652/2148342186.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cc_institution[binary_cols] = cc_institution[binary_cols].applymap(lambda x: 1 if x == "Yes" else 0)


**Collapse Factor Levels**

In [ ]:
cc_institution.city.value_counts()

city
Chicago             40
New York            38
Brooklyn            28
Philadelphia        25
Los Angeles         24
                    ..
Clifton Forge        1
Mason                1
Clinton Township     1
Marysville           1
Lowell               1
Name: count, Length: 1787, dtype: int64

In [ ]:
# collapsing 'city'

topc = ['Chicago', 'New York', 'Brooklyn', 'Philadelphia', 'Los Angeles']
cc_institution.city = (cc_institution.city.apply(lambda x: x if x in topc else "Other")).astype('category')

cc_institution.city.value_counts()

city
Other           3643
Chicago           40
New York          38
Brooklyn          28
Philadelphia      25
Los Angeles       24
Name: count, dtype: int64

In [ ]:
cc_institution.state.value_counts()

state
California              350
New York                241
Pennsylvania            217
Texas                   207
Ohio                    185
Florida                 176
Illinois                141
North Carolina          135
Georgia                 106
Virginia                104
Missouri                103
Minnesota               101
Massachusetts           100
Tennessee                94
Michigan                 94
Washington               73
Wisconsin                71
Colorado                 70
Alabama                  69
Kentucky                 67
Indiana                  67
South Carolina           66
Arizona                  63
Kansas                   62
Iowa                     61
Louisiana                56
Oklahoma                 56
New Jersey               55
Maryland                 52
Oregon                   50
Arkansas                 48
New Mexico               40
West Virginia            39
Mississippi              36
Nebraska                 36
Connecticut   

In [ ]:
# collapsing 'state'

top = ['California', 'New York', 'Pennsylvania', 'Texas', 'Ohio']
cc_institution.state = (cc_institution.state.apply(lambda x: x if x in top else "Other")).astype('category')

cc_institution.state.value_counts()

state
Other           2598
California       350
New York         241
Pennsylvania     217
Texas            207
Ohio             185
Name: count, dtype: int64

In [ ]:
cc_institution.level.value_counts() # looks good

level
4-year    2339
2-year    1459
Name: count, dtype: int64

In [ ]:
cc_institution.basic.value_counts()

basic
Associates--Private For-profit                                                  517
Masters Colleges and Universities--larger programs                              386
Baccalaureate Colleges--Diverse Fields                                          343
Associates--Public Rural-serving Medium                                         289
Baccalaureate Colleges--Arts & Sciences                                         252
Masters Colleges and Universities--medium programs                              169
Associates--Public Rural-serving Large                                          128
Associates--Public Urban-serving Multicampus                                    125
Baccalaureate/Associates Colleges                                               124
Schools of art- music- and design                                               114
Associates--Public Rural-serving Small                                          111
Masters Colleges and Universities--smaller programs                   

In [ ]:
# collapsing 'basic'

topb = ['Associates--Private For-profit',
        'Masters Colleges and Universities--larger programs',
        'Baccalaureate Colleges--Diverse Fields', 
        'Associates--Public Rural-serving Medium',
        'Baccalaureate Colleges--Arts & Sciences']
cc_institution.basic = (cc_institution.basic.apply(lambda x: x if x in topb else "Other")).astype('category')

cc_institution.basic.value_counts()

basic
Other                                                 2011
Associates--Private For-profit                         517
Masters Colleges and Universities--larger programs     386
Baccalaureate Colleges--Diverse Fields                 343
Associates--Public Rural-serving Medium                289
Baccalaureate Colleges--Arts & Sciences                252
Name: count, dtype: int64

**Normalization**

In [ ]:
# normalizing

continuous_vars = [
    'long_x', 'lat_y', 'student_count', 'awards_per_value', 'awards_per_state_value', 
    'awards_per_natl_value', 'exp_award_value', 'exp_award_state_value', 
    'exp_award_natl_value', 'exp_award_percentile', 'fte_value', 'fte_percentile', 
    'ft_pct', 'med_sat_value', 'med_sat_percentile', 'aid_value', 'aid_percentile', 
    'endow_value', 'endow_percentile', 'grad_100_value', 'grad_100_percentile', 
    'grad_150_value', 'grad_150_percentile', 'pell_value', 'pell_percentile', 
    'retain_value', 'retain_percentile', 'ft_fac_value', 'ft_fac_percentile', 
    'vsa_grad_after4_first', 'vsa_grad_elsewhere_after4_first', 'vsa_enroll_after4_first', 
    'vsa_enroll_elsewhere_after4_first', 'vsa_grad_after6_first', 'vsa_grad_elsewhere_after6_first', 
    'vsa_enroll_after6_first', 'vsa_enroll_elsewhere_after6_first', 'vsa_grad_after4_transfer', 
    'vsa_grad_elsewhere_after4_transfer', 'vsa_enroll_after4_transfer', 'vsa_enroll_elsewhere_after4_transfer', 
    'vsa_grad_after6_transfer', 'vsa_grad_elsewhere_after6_transfer', 'vsa_enroll_after6_transfer', 
    'vsa_enroll_elsewhere_after6_transfer', 'cohort_size', 'counted_percent'
]


scaler = MinMaxScaler()
cc_institution[continuous_vars] = scaler.fit_transform(cc_institution[continuous_vars])

cc_institution[continuous_vars].describe()

,long_x,lat_y,student_count,awards_per_value,awards_per_state_value,awards_per_natl_value,exp_award_value,exp_award_state_value,exp_award_natl_value,exp_award_percentile,...,vsa_grad_after4_transfer,vsa_grad_elsewhere_after4_transfer,vsa_enroll_after4_transfer,vsa_enroll_elsewhere_after4_transfer,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,cohort_size,counted_percent
count,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,...,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,3467.000000,3372.000000
mean,0.741873,0.357081,0.026176,0.167288,0.346484,0.367119,0.012320,0.277221,0.469369,0.499200,...,0.581667,0.459311,0.189670,0.341575,0.649052,0.456242,0.174505,0.277447,0.040504,0.384506
std,0.170197,0.097633,0.043362,0.077426,0.111514,0.292666,0.020340,0.188615,0.388565,0.290547,...,0.188206,0.083494,0.096437,0.154939,0.183300,0.113200,0.125523,0.140313,0.059267,0.170166
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.673895,0.282807,0.003281,0.122538,0.283951,0.306748,0.006117,0.133036,0.168790,0.250000,...,0.490909,0.418981,0.129070,0.245059,0.571669,0.402439,0.088663,0.197183,0.008011,0.254000
50%,0.787466,0.373626,0.010413,0.151714,0.335097,0.368098,0.009575,0.236110,0.181568,0.500000,...,0.592045,0.458333,0.182946,0.328063,0.666293,0.456098,0.156977,0.267606,0.021876,0.390667
75%,0.861936,0.424860,0.030267,0.189643,0.370370,0.496933,0.014564,0.379348,1.000000,0.750000,...,0.697443,0.490741,0.233721,0.426877,0.762318,0.509756,0.226744,0.338028,0.048158,0.518000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**One-hot Encoding**

In [ ]:
# one-hot encoding 

category_list = list(cc_institution.select_dtypes('category'))

cc_1h = pd.get_dummies(cc_institution, columns = category_list) 

cc_1h.describe()

,unitid,hbcu,flagship,long_x,lat_y,student_count,awards_per_value,awards_per_state_value,awards_per_natl_value,exp_award_value,...,vsa_enroll_after4_transfer,vsa_enroll_elsewhere_after4_transfer,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,state_sector_ct,carnegie_ct,cohort_size,counted_percent
count,3798.000000,3798.0,3798.0,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,...,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,3798.000000,3798.000000,3467.000000,3372.000000
mean,226063.790416,0.0,0.0,0.741873,0.357081,0.026176,0.167288,0.346484,0.367119,0.012320,...,0.189670,0.341575,0.649052,0.456242,0.174505,0.277447,34.284360,232.557662,0.040504,0.384506
std,105065.777685,0.0,0.0,0.170197,0.097633,0.043362,0.077426,0.111514,0.292666,0.020340,...,0.096437,0.154939,0.183300,0.113200,0.125523,0.140313,28.311303,156.960653,0.059267,0.170166
min,100654.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,155063.250000,0.0,0.0,0.673895,0.282807,0.003281,0.122538,0.283951,0.306748,0.006117,...,0.129070,0.245059,0.571669,0.402439,0.088663,0.197183,14.000000,106.000000,0.008011,0.254000
50%,198075.000000,0.0,0.0,0.787466,0.373626,0.010413,0.151714,0.335097,0.368098,0.009575,...,0.182946,0.328063,0.666293,0.456098,0.156977,0.267606,25.000000,169.000000,0.021876,0.390667
75%,233745.000000,0.0,0.0,0.861936,0.424860,0.030267,0.189643,0.370370,0.496933,0.014564,...,0.233721,0.426877,0.762318,0.509756,0.226744,0.338028,48.000000,343.000000,0.048158,0.518000
max,475291.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,116.000000,517.000000,1.000000,1.000000


**Baseline/Prevalence**

In [ ]:
cc_1h.retain_value.describe()
# upper quartile above 0.78

count    3535.000000
mean        0.662319
std         0.170339
min         0.000000
25%         0.561000
50%         0.669000
75%         0.781000
max         1.000000
Name: retain_value, dtype: float64

In [ ]:
cc_1h.boxplot(column='retain_value', vert=False, grid=False)

<Axes: >

In [ ]:
# adding as predictor 

cc_1h['retain_value_p'] = pd.cut(cc_1h.retain_value, bins = [-1,0.78,1], labels =[0,1]) # distribution properties; start, cut, stop

cc_1h # new column retain_value_p is now binary based on if the continuous value is above 0.78 or not

,unitid,chronname,hbcu,flagship,long_x,lat_y,site,student_count,awards_per_value,awards_per_state_value,...,control_Private for-profit,control_Private not-for-profit,control_Public,basic_Associates--Private For-profit,basic_Associates--Public Rural-serving Medium,basic_Baccalaureate Colleges--Arts & Sciences,basic_Baccalaureate Colleges--Diverse Fields,basic_Masters Colleges and Universities--larger programs,basic_Other,retain_value_p
0,100654,Alabama A&M University,0,0,0.790292,0.292177,www.aamu.edu/,0.023677,0.099927,0.275132,...,False,False,True,False,False,False,False,True,False,0
1,100663,University of Alabama at Birmingham,0,0,0.787680,0.267361,www.uab.edu,0.067476,0.148796,0.275132,...,False,False,True,False,False,False,False,False,True,1
2,100690,Amridge University,0,0,0.794572,0.245286,www.amridgeuniversity.edu,0.001758,0.214442,0.257496,...,False,True,False,False,False,True,False,False,False,0
3,100706,University of Alabama at Huntsville,0,0,0.789533,0.291004,www.uah.edu,0.033347,0.148796,0.275132,...,False,False,True,False,False,False,False,False,True,1
4,100724,Alabama State University,0,0,0.793252,0.245319,www.alasu.edu/email/index.aspx,0.031348,0.080963,0.275132,...,False,False,True,False,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,461528,Grace College of Divinity,0,0,0.872837,0.297512,www.gcd.edu,0.000511,0.187454,0.320988,...,False,True,False,False,False,False,False,False,True,0
3794,462354,John Paul the Great Catholic University,0,0,0.459168,0.259974,www.jpcatholic.com/,0.000688,0.117433,0.338624,...,False,True,False,False,False,False,False,False,True,0
3795,466921,Chamberlain College of Nursing-Missouri,0,0,0.748321,0.368084,www.chamberlain.edu,0.002733,0.398249,0.507937,...,True,False,False,False,False,False,False,False,True,0
3796,474881,Minneapolis Media Institute,0,0,0.716903,0.487486,www.mediainstitute.edu/minneapolis,0.000729,0.235594,0.613757,...,True,False,False,False,False,False,False,False,True,1


In [ ]:
# check prevalence
prevalence = cc_1h.retain_value_p.value_counts()[1]/len(cc_1h.retain_value_p)
# value_count()[1] pulls the count of '1' values in the column (values above .78)

prevalence 

# close to .25 because it's technically the top 25 percentile 

np.float64(0.23433385992627698)

**Dropping Variables & Partitioning**

In [ ]:
# dividing data into three parts: training, tuning, and test
# cleaning up dataset

cc_df = cc_1h.drop(['unitid','chronname', 'site', 'nicknames', 'long_x', 'lat_y'],axis=1).dropna(subset=['retain_value_p']) # creating a new dataframe so we don't delete these columns from our working environment. 
print(cc_df.retain_value_p.value_counts())
print(len(cc_df))

retain_value_p
0    2645
1     890
Name: count, dtype: int64
3535


In [ ]:
# partitioning 
Train, Test = train_test_split(cc_df,  train_size = .70, stratify = cc_df.retain_value_p) 

print(Train.shape)
print(Test.shape)

(2474, 75)
(1061, 75)


In [ ]:
# tuning set
Tune, Test = train_test_split(Test,  train_size = .5, stratify= Test.retain_value_p)

In [ ]:
# check the prevalence of Train
print(Train.retain_value_p.value_counts())
print(623/(1851+623))

retain_value_p
0    1851
1     623
Name: count, dtype: int64
0.25181891673403395


In [ ]:
# check the prevalence of Tune
print(Tune.retain_value_p.value_counts())
print(133/(397+133))

retain_value_p
0    397
1    133
Name: count, dtype: int64
0.2509433962264151


**Complete Function**

In [ ]:
def preprocess_cc(df, train_size=0.70, tune_size=0.50, retain_threshold=0.78):
    # categorical columns to 'category' dtype
    categorical_cols = ["city", "state", "level", "control", "basic"]
    df[categorical_cols] = df[categorical_cols].astype("category")
    
    # binary variables to boolean (1/0)
    binary_cols = ["hbcu", "flagship"]
    df[binary_cols] = df[binary_cols].fillna("No") 
    df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if x == "Yes" else 0)
    
    # split 'counted_pct' column if it exists
    if 'counted_pct' in df.columns:
        df[['counted_percent', 'counted_pct_category']] = df['counted_pct'].str.split('|', expand=True) 
        df['counted_percent'] = pd.to_numeric(df['counted_percent'], errors='coerce')
    
    # vsa_year' to integer, filling NaN with 0
    if 'vsa_year' in df.columns:
        df['vsa_year'] = df['vsa_year'].fillna(0).astype(int)
    
    # drop unnecessary columns
    drop_cols = ["index", "similar", "counted_pct", "unitid", "chronname", "site", "nicknames", "long_x", "lat_y"]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')
    
    # collapsing factor levels for select categorical variables
    top_cities = ['Chicago', 'New York', 'Brooklyn', 'Philadelphia', 'Los Angeles']
    df.city = df.city.apply(lambda x: x if x in top_cities else "Other").astype('category')
    
    top_states = ['California', 'New York', 'Pennsylvania', 'Texas', 'Ohio']
    df.state = df.state.apply(lambda x: x if x in top_states else "Other").astype('category')
    
    top_basic = [
        'Associates--Private For-profit',
        'Masters Colleges and Universities--larger programs',
        'Baccalaureate Colleges--Diverse Fields',
        'Associates--Public Rural-serving Medium',
        'Baccalaureate Colleges--Arts & Sciences'
    ]
    df.basic = df.basic.apply(lambda x: x if x in top_basic else "Other").astype('category')
    
    # normalizing continuous variables
    continuous_vars = ['retain_value', 'counted_percent']
    scaler = MinMaxScaler()
    df[continuous_vars] = scaler.fit_transform(df[continuous_vars])
    
    # one-hot encode categorical variables
    category_list = list(df.select_dtypes('category'))
    df = pd.get_dummies(df, columns=category_list)
    
    # bin 'retain_value' into categories
    df['retain_value_p'] = pd.cut(df.retain_value, bins=[-1, retain_threshold, 1], labels=[0, 1])
    
    # drop rows missing 'retain_value_p'
    df.dropna(subset=['retain_value_p'], inplace=True)
    
    # partition (with random_state for reproducibility)
    Train, Test = train_test_split(df, train_size=train_size, stratify=df.retain_value_p, random_state=42)
    Tune, Test = train_test_split(Test, train_size=tune_size, stratify=Test.retain_value_p, random_state=42)
    
    return Train, Tune, Test

# testing the function
Train, Tune, Test = preprocess_cc(cc_institution)
    

/tmp/ipykernel_66652/2461192784.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if x == "Yes" else 0)


**Using `DecisionTreeClassifier`**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score

# encoding categorical variables
Train_encoded = pd.get_dummies(Train) 
Test_encoded = pd.get_dummies(Test)

# checking that both Train and Test have the same columns
Test_encoded = Test_encoded.reindex(columns=Train_encoded.columns, fill_value=0)

# convert one-hot encoded target back to a single column because DecisionTreeClassifier only takes a single column
y_train = Train_encoded[['retain_value_p_0', 'retain_value_p_1']].idxmax(axis=1).str[-1].astype(int)
y_test = Test_encoded[['retain_value_p_0', 'retain_value_p_1']].idxmax(axis=1).str[-1].astype(int)

# dropping one-hot encoded target columns from features
X_train = Train_encoded.drop(columns=['retain_value_p_0', 'retain_value_p_1'])
X_test = Test_encoded.drop(columns=['retain_value_p_0', 'retain_value_p_1'])

# DecisionTreeClassifier
dtree = DecisionTreeClassifier()

# fit model on training data
dtree.fit(X_train, y_train)

# predict on test data
y_pred_dtree = dtree.predict(X_test)

# calculating precision
precision = precision_score(y_test, y_pred_dtree)

print(f'Precision: {precision}') 

# The precision score (perfect, in this case) is telling us how many of the instances predicted as having 
# an upper quartile retain value, actually had an upper quartile retain value. 

Precision: 1.0


#### Job Placement Data

In [ ]:
# Job Placement Data 

job_placement = pd.read_csv("https://raw.githubusercontent.com/DG1606/CMS-R-2020/master/Placement_Data_Full_Class.csv")
job_placement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sl_no           215 non-null    int64  
 1   gender          215 non-null    object 
 2   ssc_p           215 non-null    float64
 3   ssc_b           215 non-null    object 
 4   hsc_p           215 non-null    float64
 5   hsc_b           215 non-null    object 
 6   hsc_s           215 non-null    object 
 7   degree_p        215 non-null    float64
 8   degree_t        215 non-null    object 
 9   workex          215 non-null    object 
 10  etest_p         215 non-null    float64
 11  specialisation  215 non-null    object 
 12  mba_p           215 non-null    float64
 13  status          215 non-null    object 
 14  salary          148 non-null    float64
dtypes: float64(6), int64(1), object(8)
memory usage: 25.3+ KB


**Correcting Variable Types**

In [ ]:
# categorical variables to 'category' dtype
jcategorical_cols = ["gender", "hsc_s", "degree_t", "specialisation", "ssc_b", "hsc_b", ]
job_placement[jcategorical_cols] = job_placement[jcategorical_cols].astype("category")

# binary variables to boolean (1/0)
jbinary_cols = ["workex", "status"]
job_placement[jbinary_cols] = job_placement[jbinary_cols].fillna("No") 
job_placement[jbinary_cols] = job_placement[jbinary_cols].applymap(lambda x: 1 if x == "Yes" else 0)

# dropping unnecessary columns
job_placement.drop(columns=["sl_no"], inplace=True, errors='ignore')

# updated
job_placement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   gender          215 non-null    category
 1   ssc_p           215 non-null    float64 
 2   ssc_b           215 non-null    category
 3   hsc_p           215 non-null    float64 
 4   hsc_b           215 non-null    category
 5   hsc_s           215 non-null    category
 6   degree_p        215 non-null    float64 
 7   degree_t        215 non-null    category
 8   workex          215 non-null    int64   
 9   etest_p         215 non-null    float64 
 10  specialisation  215 non-null    category
 11  mba_p           215 non-null    float64 
 12  status          215 non-null    int64   
 13  salary          148 non-null    float64 
dtypes: category(6), float64(6), int64(2)
memory usage: 15.6 KB


/tmp/ipykernel_66652/1856995601.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  job_placement[jbinary_cols] = job_placement[jbinary_cols].applymap(lambda x: 1 if x == "Yes" else 0)


**Collapse Factor Variables**

In [ ]:
job_placement.gender.value_counts() # looks good

gender
M    139
F     76
Name: count, dtype: int64

In [ ]:
job_placement.hsc_s.value_counts()

hsc_s
Commerce    113
Science      91
Arts         11
Name: count, dtype: int64

In [ ]:
job_placement.degree_t.value_counts()

degree_t
Comm&Mgmt    145
Sci&Tech      59
Others        11
Name: count, dtype: int64

In [ ]:
job_placement.specialisation.value_counts()

specialisation
Mkt&Fin    120
Mkt&HR      95
Name: count, dtype: int64

In [ ]:
job_placement.ssc_b.value_counts()

ssc_b
Central    116
Others      99
Name: count, dtype: int64

Didn't need to collapse any variables! 

**Normalization**

In [ ]:
# normalizing continuous variables

abc = list(job_placement.select_dtypes('number')) #select function to find the numeric variables and create a list  

job_placement[abc] = MinMaxScaler().fit_transform(job_placement[abc])

job_placement

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,M,0.538240,Others,0.889621,Others,Commerce,0.195122,Sci&Tech,0.0,0.104167,Mkt&HR,0.284483,0.0,0.094595
1,M,0.792414,Central,0.680890,Others,Science,0.670244,Sci&Tech,1.0,0.760417,Mkt&Fin,0.564843,0.0,0.000000
2,M,0.497011,Central,0.510708,Central,Arts,0.341463,Comm&Mgmt,0.0,0.520833,Mkt&Fin,0.247001,0.0,0.067568
3,M,0.311482,Central,0.247117,Central,Science,0.048780,Sci&Tech,0.0,0.333333,Mkt&HR,0.308096,0.0,NaN
4,M,0.925788,Central,0.602965,Central,Commerce,0.568293,Comm&Mgmt,0.0,0.975000,Mkt&Fin,0.160795,0.0,0.304054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,0.818594,Others,0.741351,Others,Commerce,0.673171,Comm&Mgmt,0.0,0.854167,Mkt&Fin,0.872564,0.0,0.270270
211,M,0.352711,Others,0.378913,Others,Science,0.536585,Sci&Tech,0.0,0.500000,Mkt&Fin,0.090330,0.0,0.101351
212,M,0.538240,Others,0.494234,Others,Commerce,0.560976,Comm&Mgmt,1.0,0.187500,Mkt&Fin,0.693778,0.0,0.128378
213,F,0.682540,Others,0.477759,Others,Commerce,0.195122,Comm&Mgmt,0.0,0.416667,Mkt&HR,0.338081,0.0,0.005405


**One-hot Encoding**

In [ ]:
category_list1 = list(job_placement.select_dtypes('category').columns)
category_list1.remove('gender')  # exclude the 'gender' column from encoding

job_1h = pd.get_dummies(job_placement, columns=category_list1)

job_1h

,gender,ssc_p,hsc_p,degree_p,workex,etest_p,mba_p,status,salary,ssc_b_Central,...,hsc_b_Central,hsc_b_Others,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,specialisation_Mkt&Fin,specialisation_Mkt&HR
0,M,0.538240,0.889621,0.195122,0.0,0.104167,0.284483,0.0,0.094595,False,...,False,True,False,True,False,False,False,True,False,True
1,M,0.792414,0.680890,0.670244,1.0,0.760417,0.564843,0.0,0.000000,True,...,False,True,False,False,True,False,False,True,True,False
2,M,0.497011,0.510708,0.341463,0.0,0.520833,0.247001,0.0,0.067568,True,...,True,False,True,False,False,True,False,False,True,False
3,M,0.311482,0.247117,0.048780,0.0,0.333333,0.308096,0.0,NaN,True,...,True,False,False,False,True,False,False,True,False,True
4,M,0.925788,0.602965,0.568293,0.0,0.975000,0.160795,0.0,0.304054,True,...,True,False,False,True,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,0.818594,0.741351,0.673171,0.0,0.854167,0.872564,0.0,0.270270,False,...,False,True,False,True,False,True,False,False,True,False
211,M,0.352711,0.378913,0.536585,0.0,0.500000,0.090330,0.0,0.101351,False,...,False,True,False,False,True,False,False,True,True,False
212,M,0.538240,0.494234,0.560976,1.0,0.187500,0.693778,0.0,0.128378,False,...,False,True,False,True,False,True,False,False,True,False
213,F,0.682540,0.477759,0.195122,0.0,0.416667,0.338081,0.0,0.005405,False,...,False,True,False,True,False,True,False,False,False,True


**Baseline/Prevalence** - not needed because target variable is categorical

**Dropping Variables & Partitioning**

In [ ]:
# dividing data into three parts: training, tuning, and test
# cleaning up dataset

job_p = job_1h.drop(['ssc_b_Central','ssc_b_Others','hsc_b_Central', 'hsc_b_Others'],axis=1) # creating a new dataframe so we don't delete these columns from our working environment. 
print(job_1h.gender.value_counts())
print(len(job_p))

gender
M    139
F     76
Name: count, dtype: int64
215


In [ ]:
# partitioning 
Train1, Test1 = train_test_split(job_p,  train_size = .70, stratify = job_p.gender) 

print(Train.shape)
print(Test.shape)

(2474, 75)
(531, 75)


In [ ]:
# tuning set
Tune1, Test1 = train_test_split(Test1,  train_size = .5, stratify= Test1.gender)

In [ ]:
# check the prevalence of Train
print(Train1.gender.value_counts())
print(53/(97+53))

gender
M    97
F    53
Name: count, dtype: int64
0.35333333333333333


In [ ]:
# check the prevalence of Train
print(Tune1.gender.value_counts())
print(11/(21+11))

gender
M    21
F    11
Name: count, dtype: int64
0.34375


**Complete Function**

In [ ]:
def preprocess_job_p(df, train_size=0.70, tune_size=0.50):
    # categorical columns to 'category' dtype
    categorical_cols = ["gender", "hsc_s", "degree_t", "specialisation", "ssc_b", "hsc_b"]
    df[categorical_cols] = df[categorical_cols].astype("category")
    
    # binary variables to boolean (1/0)
    binary_cols = ["workex", "status"]
    df[binary_cols] = df[binary_cols].fillna("No") 
    df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if x == "Yes" else 0)
    
    # unnecessary columns
    df.drop(columns=["sl_no"], inplace=True, errors='ignore')
    
    # normalize continuous variables
    numeric_cols = list(df.select_dtypes('number'))
    df[numeric_cols] = MinMaxScaler().fit_transform(df[numeric_cols])
    
    # one-hot encode categorical variables (excluding 'gender')
    category_list = list(df.select_dtypes('category').columns)
    category_list.remove('gender') 
    df = pd.get_dummies(df, columns=category_list)
    
    # drop unnecessary columns
    df = df.drop(['ssc_b_Central', 'ssc_b_Others', 'hsc_b_Central', 'hsc_b_Others'], axis=1, errors='ignore')
    
    # partition
    Train, Test = train_test_split(df, train_size=train_size, stratify=df.gender, random_state=42)
    Tune, Test = train_test_split(Test, train_size=tune_size, stratify=Test.gender, random_state=42)
    
    return Train, Tune, Test

# testing the function
Train, Tune, Test = preprocess_job_p(job_placement)

/tmp/ipykernel_66652/3651350925.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if x == "Yes" else 0)


**Using `DecisionTreeClassifier`**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder

# encode categorical variables for features (excluding 'gender')
Train1_encoded = pd.get_dummies(Train1.drop(columns=['gender']))
Test1_encoded = pd.get_dummies(Test1.drop(columns=['gender']))

# checking Train1_encoded and Test1_encoded have the same columns
Test1_encoded = Test1_encoded.reindex(columns=Train1_encoded.columns, fill_value=0)

# encode target variable ('gender') using LabelEncoder - received help with this from ChatGPT
le = LabelEncoder()
y1_train = le.fit_transform(Train1['gender'])  # Converts 'M'/'F' to 0/1
y1_test = le.transform(Test1['gender'])

# features
X1_train = Train1_encoded
X1_test = Test1_encoded

# DecisionTreeClassifier
dtree = DecisionTreeClassifier()

# model on training data
dtree.fit(X1_train, y1_train)

# predict on test data
y1_pred_dtree1 = dtree.predict(X1_test)

# calculating precision
precision = precision_score(y1_test, y1_pred_dtree1, pos_label=le.transform(['M'])[0])

print(f'Precision: {precision}')

# The precision score is telling us how many of the instances predicted as "Male" (M) were actually Male.

Precision: 0.6521739130434783
